**NOTE**: This is work in progress code for flightgear integration. Use it at your own risk.

In [ ]:
import numpy as np

import sys
sys.path.append("..")
sys.path.append("csaf/core")

import csaf
import matplotlib.pyplot as plt
from csaf_f16 import fgnetfdm, fgintruder

from csaf_f16.goals import AcasSimpleCollideWithBalloonGoal
from csaf_f16.goals import AcasSimpleCollideAvoidBalloonGoal
from csaf_f16.goals import AcasShieldAvoidBalloonGoal
from csaf_f16.goals import AcasAirspeedAvoidNoBalloonGoal
from csaf_f16.goals import AcasAirspeedCollideNoBalloonGoal
from csaf_f16.ngoals import AcasAirportCollideGoal
from csaf_f16.ngoals import AcasRejoinCollideGoal
from csaf_f16.ngoals import AcasHeadOnCollideGoal
from csaf_f16.ngoals import AcasRejoinScenario

from csaf_f16.acas import AcasScenarioViewer

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
def closest_point(trajs):
    # Find the point of closest contact
    ownship = trajs['plant']['states']
    intruder = trajs['intruder_plant']['states']

    d_min = np.inf
    d_idx = 0
    for idx in range(0,min(len(ownship),len(intruder))):
        pn_o = ownship[idx][9]
        pe_o = ownship[idx][10]
        pn_i = intruder[idx][9]
        pe_i = intruder[idx][10]
        d = np.sqrt((pn_o -pn_i)**2 + (pe_o - pe_i)**2)
        if d < d_min:
            d_min = d
            d_idx = idx

    pn_0 = ownship[d_idx][9]
    pe_0 = ownship[d_idx][10]
    print(f"D_min={d_min}, idx={d_idx}, pn_0 = {pn_0},pe_0={pe_0}")
    return pn_0, pe_0

def get_camera(trajs, pn_0, pe_0):
    # Place camera
    import copy
    camera = copy.deepcopy(trajs['balloon'])
    for state in camera.states:
        state[9] = pn_0
        state[10] = pe_0
    trajs['camera'] = camera
    return trajs

In [ ]:
f16a = fgnetfdm.FGNetFDM(h0_m=1500)
f16b = fgintruder.FGIntruder(callsign='F-16B', model_path='Aircraft/f16/Models/F-16.xml', fallback_model_index=516)
#balloon = fgintruder.FGIntruder(callsign='Balloon', model_path='Aircraft/Hot-Air-Balloon/Models/Hot-Air-Balloon.xml', fallback_model_index=702)
#camera = fgintruder.FGIntruder(callsign='Camera', model_path='Aircraft/ufo/Models/ufo.xml', fallback_model_index=712)

In [ ]:
f16a.start()
f16b.start()
#balloon.start()
#camera.start()

In [ ]:
#g = AcasSimpleCollideWithBalloonGoal()
#g = AcasSimpleCollideAvoidBalloonGoal()
#g = AcasShieldAvoidBalloonGoal()
#g = AcasAirspeedAvoidNoBalloonGoal()
#g = AcasAirspeedCollideNoBalloonGoal()

#g = AcasHeadOnCollideGoal()
#g = AcasRejoinCollideGoal()
#g.scenario_type = AcasRejoinScenario
g = AcasAirportCollideGoal()

sys = g.scenario_type().generate_system(g.fixed_configurations[0])
trajs = sys.simulate_tspan((0.0, 27.0))
#trajs, p = g.run_sim(g.fixed_configurations[0])

scen = g.scenario_type()
viewer = AcasScenarioViewer(trajs, scen)
fig, ax = viewer.summary_plot()
ax.set_xlim(-10000, 10000)
ax.set_ylim(0, 25000)
plt.show()

#pn_0,pe_0 = closest_point(trajs)
#trajs = get_camera(trajs, pn_0, pe_0)



# Properly prepare the intruders
#f16a.set_trajs(trajs['plant'],trajs['llc'])
#f16b.set_trajs(trajs['intruder_plant'],trajs['intruder_llc'])
# LLC for balloon doesn't matter
#balloon.set_trajs(trajs['balloon'],trajs['llc'])
#camera.set_trajs(trajs['camera'],trajs['llc'])



In [ ]:
f16a.reset()
f16b.reset()
#balloon.reset()
#camera.reset()

In [ ]:
# Run the simulation
f16a.simulate(delta_t=0.01,speed=1.0)
f16b.simulate(delta_t=1.5,speed=1.0)

#balloon.simulate(delta_t=0.5,speed=0.9)
#camera.simulate(delta_t=0.01,speed=0.9)

In [ ]:
f16a.stop()
f16b.stop()
#balloon.stop()
#camera.stop()

In [ ]:
trajs['intruder_plant'].times